<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/6_1_both_models_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [176]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from imblearn.under_sampling import RandomUnderSampler
from google.colab import drive


In [177]:

# התחברות ל-Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_id = "/בית המשפט - קבצים שנבדקו/full_data_BOTH_models_with_predictions.xlsx"

file_path = f"/content/drive/My Drive"
# /content/drive/MyDrive/full_rap_df_cleaned.xlsx

raap = pd.read_excel(file_path+"/full_rap_df_cleaned.xlsx")
data = pd.read_excel(file_path+folder_id)

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# EDA


In [ ]:
data.columns

In [ ]:
data['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()
data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'בקשה אחרת') | (data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'דלמטה מיוחד')].shape[0]

In [ ]:
data = data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'בקשה אחרת') & (data['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'דלמטה מיוחד')]

In [ ]:
check_data = data[[
       'בקשה לרשות ערעור התקבלה?',
       'הערעור התקבל?']]

In [ ]:
check_data[check_data['בקשה לרשות ערעור התקבלה?'].isnull()]

In [ ]:
check_data[check_data[ 'הערעור התקבל?'].isnull()]

In [ ]:
clean_data = data.dropna(subset = ['בקשה לרשות ערעור התקבלה?']).copy()

In [ ]:
# clean_data[['גוף המסמך', 'מספר הליך', 'שם הליך',
#        'שנת פתיחת ההליך', 'predicted_probability',
#        'prediction_original_model',
#        'prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?', 'prediction_second_model',
#        'second_model_predicted_probability',
#        'prediction_second_model_adjusted\nערעור התקבל?',
#        'בקשה לרשות ערעור התקבלה?',
#        'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
#        'הערעור התקבל?', 'הערות תוצאת ערעור', 'הערות ']]

In [ ]:
 clean_data['בקשה לרשות ערעור התקבלה?'].unique()

In [ ]:
clean_data["prediction_second_model_adjusted\nערעור התקבל?"].unique()

In [ ]:
clean_data = data[(data['בקשה לרשות ערעור התקבלה?'] != 'אחר')]

In [ ]:
clean_data.dropna(subset=["בקשה לרשות ערעור התקבלה?"],inplace=True)

In [ ]:
clean_data['binary_outcome_for_first_classifier'] = clean_data['בקשה לרשות ערעור התקבלה?'].apply(lambda x: 0 if x == 'נדחה' else 1)

In [ ]:
clean_data['binary_outcome_for_first_classifier'].value_counts()

In [ ]:
clean_data

In [ ]:
clean_data.columns

In [ ]:
clean_data["בקשה לרשות ערעור התקבלה?"].isnull().sum()

In [ ]:

def create_CM(data, labeled_col, predicted_col):
    # יצירת confusion matrix
    cm = confusion_matrix(data[labeled_col], data[predicted_col])

    # הצגת תוצאות confusion matrix
    print("Confusion Matrix:")
    print(cm)

    # ויזואליזציה של confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    if labeled_col.find("threshold") != -1:
        plt.title("Confusion Matrix for Threshold")
    plt.show()

    # חישוב Accuracy ו-F1 Score
    accuracy = accuracy_score(data[labeled_col], data[predicted_col])
    f1 = f1_score(data[labeled_col], data[predicted_col])

    # הצגת Accuracy ו-F1 Score
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1 Score: {f1:.2f}")






In [ ]:
def create_CM_and_False_DF(data, labeled_col, predicted_col, probability_col):
    """
    Creates a confusion matrix, calculates accuracy and F1 score, visualizes the matrix,
    and returns a DataFrame of errors (FN and FP).

    Parameters:
    - data (DataFrame): The DataFrame containing labeled, predicted, and probability columns.
    - labeled_col (str): Column name of the true labels.
    - predicted_col (str): Column name of the predicted labels.
    - probability_col (str): Column name of the prediction probabilities.

    Returns:
    - DataFrame: A DataFrame of errors (FN and FP) containing "מספר הליך", "גוף המסמך", "Error Type", and "Prediction Probability".
    """
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

    # יצירת confusion matrix
    cm = confusion_matrix(data[labeled_col], data[predicted_col])

    # הצגת תוצאות confusion matrix
    print("Confusion Matrix:")
    print(cm)

    # ויזואליזציה של confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

    # חישוב Accuracy ו-F1 Score
    accuracy = accuracy_score(data[labeled_col], data[predicted_col])
    f1 = f1_score(data[labeled_col], data[predicted_col])

    # הצגת Accuracy ו-F1 Score
    print(f"Accuracy: {accuracy:.2f}")
    print(f"F1 Score: {f1:.2f}")

    # False Negatives: הערך בפועל 1 אך סווג כ-0
    fn_table = data[(data[labeled_col] == 1) & (data[predicted_col] == 0)][["מספר הליך", "גוף המסמך", probability_col]].copy()
    fn_table["Error Type"] = "FN"  # הוספת עמודת סוג הטעות

    # False Positives: הערך בפועל 0 אך סווג כ-1
    fp_table = data[(data[labeled_col] == 0) & (data[predicted_col] == 1)][["מספר הליך", "גוף המסמך", probability_col]].copy()
    fp_table["Error Type"] = "FP"  # הוספת עמודת סוג הטעות

    # איחוד הטבלאות
    errors_table = pd.concat([fn_table, fp_table], ignore_index=True)
    errors_table.rename(columns={probability_col: "Prediction Probability"}, inplace=True)

    # החזרת הטבלה
    return errors_table


In [ ]:
clean_data

In [ ]:
first_classifier_False = create_CM_and_False_DF(clean_data,"prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?",'binary_outcome_for_first_classifier',"predicted_probability")

In [ ]:
first_classifier_False

## First classifier

In [ ]:
create_CM(clean_data,"prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?",'binary_outcome_for_first_classifier')

In [ ]:
create_CM(clean_data,"prediction_original_model",'binary_outcome_for_first_classifier')


In [ ]:
clean_data['binary_outcome_for_second_classifier'] = clean_data['הערעור התקבל?'].apply(lambda x: 0 if x == 'נדחה' else 1)

In [ ]:
y_probs = "predicted_probability"
y_pred_col = "prediction_original_model"
y_true_col = "binary_outcome_for_first_classifier"

### ניתוח הגרפים והסקת מסקנות תוך התחשבות במציאות האמיתית

#### **1. Sigmoid Curve**
הגרף של הסיגמואיד מדגים את פיזור הנתונים עבור עקומת המודל, כאשר בבדיקות הנוכחיות (דאטה מאוזן יחסית), יש ייצוג דומה לשתי המחלקות (0 ו-1). עם זאת, במציאות האמיתית, בה רוב הדגימות שייכות למחלקה 0 (מעל 90%), קיים סיכוי גבוה לטעויות מסוג False Positives (FP). לכן, בחירת Threshold גבוה יותר עשויה להיות עדיפה בעולם האמיתי, כדי לצמצם טעויות FP על חשבון ירידה מסוימת ב-TPR (True Positive Rate).

---

#### **2. ROC Curve**
ה-ROC Curve מספק מדד מצוין לביצועי המודל, ומראה ששטח ה-AUC (Area Under Curve) הוא 0.87, מה שמעיד על הפרדה טובה בין המחלקות. ה-Optimal Threshold המוצע בגרף הוא 0.75, שמאזן היטב בין TPR ל-FPR בדאטה המאוזן. יחד עם זאת, בעולם האמיתי, רוב הדגימות שייכות למחלקה 0, ולכן Threshold זה עשוי לספק איזון טוב לצמצום FP, אך עלול להוביל לעלייה ב-FN (False Negatives).

---

#### **3. Cost Function**
בגרף ה-Cost Function ניתן לראות ש-Threshold של 0.5 הוא האופטימלי מבחינת עלות עבור הדאטה המאוזן. עם זאת, במציאות שבה רוב הדגימות שייכות למחלקה 0, העלות של FN (החמצת מקרים קריטיים של מחלקה 1) עשויה להיות משמעותית מאוד. לפיכך, Threshold נמוך יותר מ-0.75 אך גבוה מ-0.5, למשל 0.67, יכול להוות פשרה טובה שמאזנת בין צמצום FP לבין שמירה על Recall גבוה עבור מחלקה 1.

---

#### **החלטה על Threshold מותאם לעולם האמיתי**
בהתחשב בניתוח הגרפים והמציאות האמיתית שבה הדאטה אינו מאוזן (עם רוב של מעל 90% למחלקה 0):
1. **Threshold של 0.67** נבחר כפשרה בין הגרפים השונים:
   - הוא מספיק גבוה כדי לצמצם FP.
   - שומר על Recall סביר, החשוב לאיתור נכון של מחלקה 1.
2. Threshold זה מתאים לשימוש גם על הדאטה המאוזן וגם על המציאות האמיתית, שבה עלות של FN היא גבוהה במיוחד.

---

#### **המלצות להמשך**
1. לבדוק את הביצועים של המודל עם Threshold של 0.67 על דאטה המדמה את המציאות האמיתית (עם רוב של מחלקה 0).
2. לעדכן את Cost Function ואת ה-Precision-Recall Curve במקרה של שינוי בתנאי העלות או בנתוני המציאות.

---

#### **סיכום**
Threshold של 0.67 מהווה פשרה חכמה ומעשית בהתחשב בדאטה המאוזן בבדיקות ובמציאות האמיתית של הדומיננטיות של מחלקה 0. הבחירה תאפשר איזון מיטבי בין צמצום FP לשימור Recall גבוה.


## Second classifier

In [ ]:
check_data = clean_data.dropna(subset=["הערעור התקבל?"])
check_data[check_data["prediction_second_model_adjusted\nערעור התקבל?"].isna()]

In [ ]:
check_data['הערעור התקבל?'].unique()

In [ ]:
check_data[check_data['הערעור התקבל?']=="אחר"]

In [ ]:
check_data.columns

In [ ]:
check_data = check_data[check_data['הערעור התקבל?']!="אחר"]

In [ ]:
check_data['הערעור התקבל?'].unique()

In [ ]:
check_data['binary_outcome_for_second_classifier'] = clean_data['הערעור התקבל?'].apply(lambda x: 0 if x == 'נדחה' else 1)

In [ ]:
check_data["prediction_second_model_adjusted\nערעור התקבל?"].isna().sum()

In [ ]:
check_data.dropna(subset=["prediction_second_model_adjusted\nערעור התקבל?"],inplace = True)

In [ ]:
check_data.dropna(subset=["prediction_second_model_adjusted\nערעור התקבל?"],inplace=True)

In [ ]:
create_CM(check_data,'prediction_second_model_adjusted\nערעור התקבל?','binary_outcome_for_second_classifier')

In [ ]:
create_CM(check_data,"prediction_second_model",'binary_outcome_for_second_classifier')

In [ ]:
check_data["new_prediction_col"] = [1 if x > 0.9 else 0 for x in check_data["second_model_predicted_probability"]]

create_CM(check_data,"new_prediction_col",'binary_outcome_for_second_classifier')

#0.89

In [ ]:
second_classifier_False = create_CM_and_False_DF(check_data,"new_prediction_col",'binary_outcome_for_second_classifier',"second_model_predicted_probability")

In [ ]:
second_classifier_False

In [ ]:
false_classification

In [ ]:
false_classification = pd.merge(
    first_classifier_False,
    second_classifier_False,
    how="outer",
    on="מספר הליך",
    suffixes=('_first', '_second')  # סיומות רק לעמודות שונות
)

# שילוב עמודת "גוף המסמך" משני ה-DataFrames לעמודה אחת
if "גוף המסמך_first" in false_classification.columns and "גוף המסמך_second" in false_classification.columns:
    false_classification["גוף המסמך"] = false_classification["גוף המסמך_first"].combine_first(false_classification["גוף המסמך_second"])
    false_classification.drop(columns=["גוף המסמך_first", "גוף המסמך_second"], inplace=True)


In [ ]:
false_classification

In [ ]:
output_file = "/content/drive/My Drive/False_classification_report.xlsx"  # עדכן את הנתיב לשמירת הקובץ
false_classification.to_excel(output_file, index=False)
print(f"Combined file saved at: {output_file}")

Conclusion

first classifier treshold for predction: 0.67
second classifier treshold for prediction: 0.9


# Re training

## First classifier

  

In [ ]:
data = clean_data[["גוף המסמך חתוך","binary_outcome_for_first_classifier"]]

# Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
#        'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
#        'prediction_original_model',
#        'prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?',
#        'גוף המסמך חתוך מסווג שני', 'prediction_second_model',
#        'second_model_predicted_probability',
#        'prediction_second_model_adjusted\nערעור התקבל?',
#        'בקשה לרשות ערעור התקבלה?',
#        'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
#        'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד',
#        'קידוד', 'הערות '],
#       dtype='object')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import joblib
import pandas as pd

# טעינת המודל והטוקנייזר
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

### Embedding

In [ ]:
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_predict, cross_validate
import joblib


# עדכון עמודת המטרה ואימבדינג לאחר חיתוך הטקסטים
y = data["binary_outcome_for_first_classifier"].values
embeddings = np.array([get_embeddings(text) for text in data["גוף המסמך חתוך"]])


### Train and fit

In [ ]:
# classifier_model_second_undersampled

In [ ]:
X = embeddings
# טוען את המודל
classifier = joblib.load("2_classifier_model_double_trained.pkl")

print("המודל נטען בהצלחה.")

# התאמת המודל לנתונים המאוזנים
classifier.fit(X, y)

# הגדרת מדדים לקרוס ולידציה
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=1),
    'recall': make_scorer(recall_score, pos_label=1),
    'f1': make_scorer(f1_score, pos_label=1)
}

# ביצוע Cross Validation
cv_results = cross_validate(classifier, X, y, cv=5, scoring=scoring)

# חיזוי תוויות על כל קפל לצורך קונפיושן מטריקס
y_pred = cross_val_predict(classifier, X, y, cv=5)
conf_matrix = confusion_matrix(y, y_pred)

# הדפסת התוצאות
print("Cross-Validation Accuracy:", cv_results['test_accuracy'])
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Cross-Validation Precision for Class 1:", cv_results['test_precision'])
print("Mean Precision for Class 1:", cv_results['test_precision'].mean())
print("Cross-Validation Recall for Class 1:", cv_results['test_recall'])
print("Mean Recall for Class 1:", cv_results['test_recall'].mean())
print("Cross-Validation F1 Score for Class 1:", cv_results['test_f1'])
print("Mean F1 Score for Class 1:", cv_results['test_f1'].mean())

tn, fp, fn, tp = conf_matrix.ravel()

# הדפסת הקונפיושן מטריקס עם תוויות
print("Confusion Matrix with Labels:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

print("\nConfusion Matrix:")
print(f"[[{tn} {fp}]")
print(f" [{fn} {tp}]]")


# --- שמירת משאבים ---
joblib.dump(classifier, '3_first_classifier_model_Triple_trained.pkl')

print("המודל והמשאבים נשמרו בהצלחה.")


המודל נטען בהצלחה.
Cross-Validation Accuracy: [0.78787879 0.90909091 0.96875    0.9375     0.875     ]
Mean Accuracy: 0.8956439393939395
Cross-Validation Precision for Class 1: [0.72727273 0.85       1.         0.94117647 0.84210526]
Mean Precision for Class 1: 0.8721108922037715
Cross-Validation Recall for Class 1: [0.94117647 1.         0.94117647 0.94117647 0.94117647]
Mean Recall for Class 1: 0.9529411764705882
Cross-Validation F1 Score for Class 1: [0.82051282 0.91891892 0.96969697 0.94117647 0.88888889]
Mean F1 Score for Class 1: 0.9078388137211668
Confusion Matrix with Labels:
True Negatives (TN): 64
False Positives (FP): 13
False Negatives (FN): 4
True Positives (TP): 81

Confusion Matrix:
[[64 13]
 [4 81]]
המודל והמשאבים נשמרו בהצלחה.


In [ ]:
# Confusion Matrix:
# [[52  2]
#  [25 83]]



162

## Second classifier


In [ ]:
clean_data.columns

Index(['level_0', 'index', 'גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'predicted_probability',
       'prediction_original_model',
       'prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?',
       'גוף המסמך חתוך מסווג שני', 'prediction_second_model',
       'second_model_predicted_probability',
       'prediction_second_model_adjusted\nערעור התקבל?',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד',
       'קידוד', 'הערות ', 'binary_outcome_for_first_classifier',
       'new_prediction_col'],
      dtype='object')

In [ ]:
data_1 = check_data[["גוף המסמך חתוך מסווג שני",'binary_outcome_for_second_classifier']]

In [ ]:
data_1["binary_outcome_for_second_classifier"].value_counts()

,count
binary_outcome_for_second_classifier,
1,71
0,9


In [ ]:
from sklearn.utils import resample
import pandas as pd

# שליפת המחלקות
class_1 = data_1[data_1['binary_outcome_for_second_classifier'] == 1]
class_0 = data_1[data_1['binary_outcome_for_second_classifier'] == 0]

# Undersampling למחלקה הדומיננטית (class 1)
class_1_downsampled = resample(class_1,
                               replace=False,  # ללא החלפה
                               n_samples=len(class_0)*4,  # מספר הדוגמאות במחלקה הקטנה
                               random_state=42)  # לשחזור

# איחוד מחדש של שתי המחלקות
balanced_data = pd.concat([class_1_downsampled, class_0])

# ערבוב הנתונים (Shuffle)
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# בדיקת התוצאה
print(balanced_data['binary_outcome_for_second_classifier'].value_counts())


binary_outcome_for_second_classifier
1    36
0     9
Name: count, dtype: int64


In [ ]:
balanced_data

,גוף המסמך חתוך מסווג שני,binary_outcome_for_second_classifier
0,"ן היום בעע""ם 8324/12 ועע""ם 8502/12. בפסק-הדין ...",0
1,", הוא אינו בגדר ענישה של הנאשמים, כי אם בגדר פ...",1
2,לבפיש גב\' ברכה וייס פסק דין השופט א\' א\' לוי...,1
3,"ב-2005, פסק הדין בערעור בבית המשפט המחוזי ובעק...",0
4,להותיר את ההכרעה בו לשיקול דעתו של בית המשפט....,1
5,"כפוף, כמובן, להוראות הדין. 28. נוכח כל האמור ל...",0
6,"רה של החברה נדחה אפוא (עע""ם 6724/10). בצד האמו...",1
7,קש (שיקרא להלן: המערער). זאת ועוד – על פי ההסכ...,1
8,מאסר המותנה בן 12 החודשים שהוטל על המבקש בחפיפ...,1
9,"תחו כאמור גם הליכי הוצאה לפועל הנ""ל בידי המשיב...",1


In [ ]:
y = balanced_data["binary_outcome_for_second_classifier"].values
embeddings = np.array([get_embeddings(text) for text in balanced_data["גוף המסמך חתוך מסווג שני"]])

In [ ]:
X = embeddings
# טוען את המודל
classifier = joblib.load("second_classifier_model.pkl")

print("המודל נטען בהצלחה.")

# התאמת המודל לנתונים המאוזנים
classifier.fit(X, y)

# הגדרת מדדים לקרוס ולידציה
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, pos_label=1),
    'recall': make_scorer(recall_score, pos_label=1),
    'f1': make_scorer(f1_score, pos_label=1)
}

# ביצוע Cross Validation
cv_results = cross_validate(classifier, X, y, cv=5, scoring=scoring)

# חיזוי תוויות על כל קפל לצורך קונפיושן מטריקס
y_pred = cross_val_predict(classifier, X, y, cv=5)
conf_matrix = confusion_matrix(y, y_pred)

# הדפסת התוצאות
print("Cross-Validation Accuracy:", cv_results['test_accuracy'])
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Cross-Validation Precision for Class 1:", cv_results['test_precision'])
print("Mean Precision for Class 1:", cv_results['test_precision'].mean())
print("Cross-Validation Recall for Class 1:", cv_results['test_recall'])
print("Mean Recall for Class 1:", cv_results['test_recall'].mean())
print("Cross-Validation F1 Score for Class 1:", cv_results['test_f1'])
print("Mean F1 Score for Class 1:", cv_results['test_f1'].mean())

tn, fp, fn, tp = conf_matrix.ravel()

# הדפסת הקונפיושן מטריקס עם תוויות
print("Confusion Matrix with Labels:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

print("\nConfusion Matrix:")
print(f"[[{tn} {fp}]")
print(f" [{fn} {tp}]]")


# --- שמירת משאבים ---
# joblib.dump(classifier, '2_second_classifier_model_double_trained.pkl')

print("המודל והמשאבים נשמרו בהצלחה.")


המודל נטען בהצלחה.
Cross-Validation Accuracy: [0.77777778 0.77777778 0.77777778 0.77777778 0.88888889]
Mean Accuracy: 0.8
Cross-Validation Precision for Class 1: [0.77777778 0.77777778 0.77777778 0.77777778 1.        ]
Mean Precision for Class 1: 0.8222222222222222
Cross-Validation Recall for Class 1: [1.    1.    1.    1.    0.875]
Mean Recall for Class 1: 0.975
Cross-Validation F1 Score for Class 1: [0.875      0.875      0.875      0.875      0.93333333]
Mean F1 Score for Class 1: 0.8866666666666667
Confusion Matrix with Labels:
True Negatives (TN): 1
False Positives (FP): 8
False Negatives (FN): 1
True Positives (TP): 35

Confusion Matrix:
[[1 8]
 [1 35]]
המודל והמשאבים נשמרו בהצלחה.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# חיזוי הסתברויות על הסט המאוזן
y_probs = classifier.predict_proba(X)[:, 1]

# שינוי הסף
new_threshold = 0.9
y_pred_adjusted = (y_probs >= new_threshold).astype(int)

# חישוב מטריקות
accuracy = accuracy_score(y, y_pred_adjusted)
precision = precision_score(y, y_pred_adjusted)
recall = recall_score(y, y_pred_adjusted)
f1 = f1_score(y, y_pred_adjusted)
conf_matrix = confusion_matrix(y, y_pred_adjusted)

# הדפסת תוצאות
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

tn, fp, fn, tp = conf_matrix.ravel()
print("\nConfusion Matrix:")
print(f"[[{tn} {fp}]")
print(f" [{fn} {tp}]]")


Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00

Confusion Matrix:
[[9 0]
 [0 36]]


Conclusion:
We will be using the "3_first_classifier_model_Triple_trained"
and the originak "second_classifier_model"


first classifier treshold for predction: 0.67
second classifier treshold for prediction: 0.9
